# Spherical ICET

In [2]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp

physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
%load_ext autoreload
%autoreload 2
%autosave 180

Autosaving every 180 seconds


In [112]:
from ICET_spherical import ICET

## init KITTI dataset -----------------------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
c1 = velo1[:,:3]
c1 = c1[c1[:,2] > -1.5] #ignore ground plane
## ------------------------------------------------------------------------------------

# c1 = np.random.randn(100,3)*tf.constant([3.,3.,1.])
# c1 = np.random.randn(2,3)*tf.constant([3.,3.,0.5])

#single distinct cluster
# c1 = np.random.randn(10,3)*tf.constant([0.1,0.1,0.1]) + tf.constant([2.,3.3,0.])

it = ICET(cloud1 = c1, fid = 30)
ViewInteractiveWidget(it.plt.window)


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [208]:
#test where points are inside spherical cell...
# print(it.cloud1_tensor_spherical)
maxtheta = tf.constant([[0.2],[0.7]])
maxr = tf.constant([[0.5],[2.]])

ans1 = tf.greater(it.cloud1_tensor_spherical[:,1], maxtheta)
# print(ans1)
ans2 = tf.less(it.cloud1_tensor_spherical[:,0], maxr)
# print(ans2)
combined = tf.Variable([ans1, ans2])
# print(combined)
ans3 = tf.math.reduce_all(combined, axis = 1)

print(ans3)

tf.Tensor(
[[False False False False  True False  True False False  True]
 [False False False False False False False False False False]], shape=(2, 10), dtype=bool)
